In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("spark://ip-172-31-22-115.ec2.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/27 11:51:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read some data

In [4]:
df_green = spark.read.parquet('data/pq/fhv/*/*')

In [5]:
df_green.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   null|                  null|
|              B01315|2019-10-05 15:13:04|2019-10-05 15:19:48|         264|          74|   null|                B01315|
|              B01984|2019-10-12 17:13:00|2019-10-12 17:40:00|         264|          75|   null|                B01984|
|              B00310|2019-10-15 10:55:04|2019-10-15 11:00:45|         264|         247|   null|                B03047|
|              B00932|2019-10-08 06:58:42|2019-10-08 07:11:11|         264|          37|   null|                B00932|
+--------------------+------------------

In [6]:
df_green.registerTempTable('fhv_data')

/home/ubuntu/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
spark.sql("""
SELECT
    dispatching_base_num,
    count(1)
FROM
    fhv_data
GROUP BY 
    dispatching_base_num
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|     B00625         |    3207|
|              B00727|    1624|
|              B02629|     326|
|              B02803|    7897|
|              B01616|     753|
|              B01351|      48|
|              B02841|     300|
|              B00369|     561|
|              B00411|    1046|
|     B00746         |      52|
|              B00789|    8591|
|              B01553|    5340|
|              B01328|    4362|
|     B01467         |    5186|
|              B01341|     815|
|              B02942|     554|
|              B02535|      22|
|              B00856|   36778|
|              B00318|    3023|
|     B01995         |    3318|
+--------------------+--------+
only showing top 20 rows



In [8]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PUlocationID AS trips_zone,
    date_trunc('day', pickup_datetime) AS trips_day, 
    COUNT(1) as trips
FROM
    fhv_data
GROUP BY
    1, 2
""")

In [9]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')